In [40]:
%config IPCompleter.greedy=True

In [32]:
#https://openpyxl.readthedocs.io/en/stable/
from openpyxl import load_workbook

In [33]:
#This program will open excel and will compare one column from all of the rows between two sheets (A and B)
#and create two new sheets.
#Each sheet will list every row from A with the rows from B under it that matched it

In [34]:
#Config Vars here:
Excel_Filename = "test.xlsx"
Name_Of_Sheet_A = "A"
Name_Of_Sheet_B = "B"
Col_To_Join_On = 2 #Number of the column, but REMEMBER TO SUBTRACT 1! If it's col 3, write 2 here.
Include_Not_Found = True

#Derived config vars. Don't touch
Name_Of_Sorted_Sheet_A = Name_Of_Sheet_A + " compared with " + Name_Of_Sheet_B

In [41]:
#Opening the files and loading the variables
wb = load_workbook(Excel_Filename)
sheet_A = wb[Name_Of_Sheet_A]
sheet_B = wb[Name_Of_Sheet_B]

sheet_A_sorted = wb.create_sheet(Name_Of_Outer_Joined_Sheet_A)


In [36]:
#Turn the WS object into list form
def worksheet_to_list(ws):
    ws_values = ws.values
    return [list(row) for row in ws_values]

In [37]:
sheet_A_list_form = worksheet_to_list(sheet_A)
sheet_B_list_form = worksheet_to_list(sheet_B)

In [38]:
#Isolate the columns that we're supposed to be matching on

#the_col_to_be_compared_in_sheet_b
col_B = [row[Col_To_Join_On] for row in sheet_B_list_form]
#the_col_to_be_compared_in_sheet_a 
col_A = [row[Col_To_Join_On] for row in sheet_A_list_form]

In [47]:
#We're going to loop through every single item in col_A and see if there's a match in colB
#We will get the indicies (aka row number) for all rows in sheet B that matched
#We will copy/extract those rows from sheet_B

#and write them to a new sheet in form:
# Row from Sheet A:
# X X X X X
# Matching Rows from Sheet B:
# Y Y Y Y Y
# Y Y Y Y Y
# Y Y Y Y Y

#A counter for which row we're on in the compared_sheet. Starts at 1
#because excel starts at one.
row_counter = 1 

#Looping through every single thing in A
for index, item in enumerate(col_A):
    #Get the index of all rows in B that contain a match to our current item
    matches = [ i for i in range(len(col_B)) if col_B[i] == item]
    print (item)
    #print the relevant row from A first
    sheet_A_sorted.cell(row = row_counter, column=1).value = item

11
12
13
14
15
16
17
18
19
20
10


In [45]:
wb.save('test.xlsx')

In [ ]:

# List of strings
listOfElems = ['Hello', 'Ok', 'is', 'Ok', 'test', 'this', 'is', 'a', 'test', 'Ok']
 
# Use List Comprehension Get indexes of all occurrences of 'Ok' in the list
indexPosList = [ i for i in range(len(listOfElems)) if listOfElems[i] == 'Ok' ]
 
print('Indexes of all occurrences of a "Ok" in the list are : ', indexPosList)